In [1]:
import torch

torch.__version__

'2.10.0+cu128'

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# from torchvision import datasets, transforms
#
# train_set = datasets.Food101(
#     transform=transforms.ToTensor(),
#     root="./data",
#     download=True,
#     split="train"
# )
#
# test_set = datasets.Food101(
#     transform=transforms.ToTensor(),
#     root="./data",
#     download=True,
#     split="test"
# )
#
# len(train_set), len(test_set)

"""So i can download and use it but i wont because it is too much. its size is slightly above from 5GB. and downloading and training such size of data is not good for learning i think"""

'So i can download and use it but i wont because it is too much. its size is slightly above from 5GB. and downloading and training such size of data is not good for learning i think'

In [4]:
import requests
import zipfile
from pathlib import Path

data_path = Path("./data")
image_path = data_path / "pizza_steak_sushi"
zip_path = data_path / "pizza_steak_sushi.zip"

if image_path.is_dir():
    print(f"{image_path} directory exists")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

    response = requests.get(
        "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
    )
    print("Downloading pizza, steak, sushi data...")

    with open(zip_path, "wb") as f:
        f.write(response.content)

    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        print("Extracting pizza, sushi, steak data...")
        zip_ref.extractall(image_path)

Did not find data/pizza_steak_sushi directory, creating one...
Extracting pizza, sushi, steak data...


In [5]:
def walk_through_dir(dir_path: Path):
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


walk_through_dir(image_path)

NameError: name 'os' is not defined

In [ ]:
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

In [ ]:
import random
from PIL import Image

random.seed(42)
image_path_list = list(image_path.glob("*/*/*.jpg"))
random_image_path = random.choice(image_path_list)
image_class = random_image_path.parent.stem

img = Image.open(random_image_path)

print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

img_as_array = np.asarray(img)
plt.figure(figsize=(10, 7))
plt.imshow(img_as_array)
plt.title(f"Image class: {image_class} | Image shape: {img_as_array.shape} -> [height, width, color_channels]")
plt.axis(False);

In [ ]:
from torchvision import transforms

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.AutoAugment(),
    transforms.ToTensor(),
])

In [ ]:
def plot_transformed_images(image_path, transform, n=3, seed=42):
    random.seed(seed)
    random_image_paths = random.sample(image_path, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(f)
            ax[0].set_title(f"Original \nSize: {f.size}")
            ax[0].axis("off")

            transformed_image = transform(f).permute(1, 2, 0)
            ax[1].imshow(transformed_image)
            ax[1].set_title(f"Transformed \nSize: {transformed_image.shape}")
            ax[1].axis("off")

            fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)

In [ ]:
plot_transformed_images(image_path_list, data_transform, n=5, seed=43)

In [ ]:
from torchvision import datasets

train_set = datasets.ImageFolder(
    root=train_dir,
    transform=data_transform,
)

test_set = datasets.ImageFolder(
    root=test_dir,
    transform=data_transform,
)

train_set, test_set, len(train_set), len(test_set)

In [ ]:
classes = train_set.classes
classes

In [ ]:
classes_dict = train_set.class_to_idx
classes_dict

In [ ]:
img, label = train_set[0][0], train_set[0][1]
print(f"Image tensor:\n{img}")
print(f"Image shape: {img.shape}")
print(f"Image datatype: {img.dtype}")
print(f"Image label: {label}")
print(f"Label datatype: {type(label)}")

In [ ]:
img_permute = img.permute(1, 2, 0)

print(f"Original shape: {img.shape} -> [color_channels, height, width]")
print(f"Image permute shape: {img_permute.shape} -> [height, width, color_channels]")

plt.figure(figsize=(10, 7))
plt.imshow(img.permute(1, 2, 0))
plt.axis("off")
plt.title(classes[label], fontsize=14);

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_set,
    batch_size=2,
    shuffle=True,
    num_workers=2,
)

test_loader = DataLoader(
    dataset=test_set,
    batch_size=4,
    num_workers=2,
    shuffle=False,
)

In [ ]:
img, label = next(iter(train_loader))
img.shape, label.shape

##### Custom dataset

In [ ]:
import os

from PIL import Image

In [ ]:
def find_classes(dir: Path):
    class_names = sorted(entry.name for entry in os.scandir(dir) if entry.is_dir())
    if not class_names:
        raise FileNotFoundError(f"No classes found in {dir}")

    class_to_idx = {cls_name: i for i, cls_name in enumerate(class_names)}
    return class_names, class_to_idx

In [ ]:
find_classes(train_dir)

In [6]:
from torch.utils.data import Dataset


class ImageFolderCustom(Dataset):
    def __init__(self, target_dir, transform=None):
        self.transform = transform
        self.paths = list(Path(target_dir).glob("*/*.jpg"))
        self.classes, self.class_to_idx = find_classes(target_dir)

    def load_image(self, index: int):
        path = self.paths[index]
        return Image.open(path)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index: int):
        img = self.load_image(index)
        class_name = self.paths[index].parent.name
        class_idx = self.class_to_idx[class_name]

        if self.transform:
            return self.transform(img), class_idx
        else:
            return img, class_idx

In [ ]:
train_transformer = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
])

test_transformer = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [ ]:
train_set_custom = ImageFolderCustom(train_dir, train_transformer)
test_set_custom = ImageFolderCustom(test_dir, test_transformer)

train_set_custom, test_set_custom

In [ ]:
train_set_custom.classes

In [ ]:
train_set_custom.class_to_idx